In [1]:
import os
import cv2
import pafy
import math
import random
import numpy as np
import datetime as dt
import tensorflow as tf
from collections import deque
import matplotlib.pyplot as plt

from moviepy.editor import *
%matplotlib inline
from sklearn.model_selection import train_test_split
from tensorflow.keras.layers import *
from tensorflow.keras.models import Sequential
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.utils import plot_model

In [2]:
seed_constant=27
np.random.seed(seed_constant)
random.seed(seed_constant)
tf.random.set_seed(seed_constant)

In [3]:
IMAGE_HEIGHT,IMAGE_WIDTH=64,64#dimensions of the frame
SEQUENCE_LENGTH=10#no owf frames of a video which would be fed
DATASET_DIR=r'C:\Users\ASUS\OneDrive\Desktop\majorproject\hockey'


In [4]:
CLASSES_LIST=["FIGHT","NOFIGHT"]

In [5]:
def frames_extraction(video_path):
    frames_list=[]
    video_reader=cv2.VideoCapture(video_path)
    video_frames_count=int(video_reader.get(cv2.CAP_PROP_FRAME_COUNT))
    
    skip_frames_window=max(int(video_frames_count/SEQUENCE_LENGTH),1)#the interval after which frrames get added
    #iterate thru frames
    for frame_counter in range(SEQUENCE_LENGTH):
        video_reader.set(cv2.CAP_PROP_POS_FRAMES,frame_counter*skip_frames_window)
        success,frame=video_reader.read()
        if not success:
            break
        resized_frame=cv2.resize(frame,(IMAGE_HEIGHT,IMAGE_WIDTH))
        normalized_frame=resized_frame/255
        frames_list.append(normalized_frame)
    video_reader.release()
   
    return frames_list

In [6]:
import os
frames2=0
def create_dataset():
    features = []
    labels = []
    video_files_paths = []
    for class_index, class_name in enumerate(CLASSES_LIST):
        print(f'Extracting Data of Class: {class_name}')
        files_list = os.listdir(os.path.join(DATASET_DIR, class_name))
        
        for file_name in files_list:
            video_file_path = os.path.join(DATASET_DIR, class_name, file_name)

            frames = frames_extraction(video_file_path)
           
            

            if len(frames) == SEQUENCE_LENGTH:

                features.append(frames)
                labels.append(class_index)
                video_files_paths.append(video_file_path)

    features = np.asarray(features)
    labels = np.array(labels)
    frames2=frames
    
    return features, labels, video_files_paths
       

In [8]:
features, labels, video_files_paths = create_dataset()


Extracting Data of Class: FIGHT
Extracting Data of Class: NOFIGHT


In [9]:
one_hot_encoded_labels = to_categorical(labels)

In [10]:
features_train, features_test, labels_train, labels_test = train_test_split(features, one_hot_encoded_labels,
                                                                            test_size = 0.25, shuffle = True,
                                                                            random_state=seed_constant
                                                                            )

In [20]:
labels_train.shape

(750, 2)

In [21]:
print(IMAGE_HEIGHT)

64


In [18]:
def create_convlstm_model():
  
    model = Sequential()

  
    model.add(ConvLSTM2D(filters = 4, kernel_size = (3, 3), activation = 'tanh',data_format = "channels_last",
                         recurrent_dropout=0.25, return_sequences=True, input_shape = (SEQUENCE_LENGTH,
                                                                                      IMAGE_HEIGHT, IMAGE_WIDTH, 3)))
    
    model.add(MaxPooling3D(pool_size=(1, 2, 2), padding='same', data_format='channels_last'))
    model.add(TimeDistributed(Dropout(0.25)))
    
    model.add(ConvLSTM2D(filters = 8, kernel_size = (3, 3), activation = 'tanh', data_format = "channels_last",
                         recurrent_dropout=0.25, return_sequences=True))
    
    model.add(MaxPooling3D(pool_size=(1, 2, 2), padding='same', data_format='channels_last'))
    model.add(TimeDistributed(Dropout(0.25)))
    
    model.add(ConvLSTM2D(filters = 14, kernel_size = (3, 3), activation = 'tanh', data_format = "channels_last",
                         recurrent_dropout=0.25, return_sequences=True))
    
    model.add(MaxPooling3D(pool_size=(1, 2, 2), padding='same', data_format='channels_last'))

   
    model.add(Flatten()) 
    
    model.add(Dense(len(CLASSES_LIST), activation = "sigmoid"))
    
    
    
    model.summary()
    return model

In [19]:
convlstm_model = create_convlstm_model()

Model: "sequential_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv_lst_m2d_6 (ConvLSTM2D)  (None, 10, 62, 62, 4)     1024      
_________________________________________________________________
max_pooling3d_6 (MaxPooling3 (None, 10, 31, 31, 4)     0         
_________________________________________________________________
time_distributed_4 (TimeDist (None, 10, 31, 31, 4)     0         
_________________________________________________________________
conv_lst_m2d_7 (ConvLSTM2D)  (None, 10, 29, 29, 8)     3488      
_________________________________________________________________
max_pooling3d_7 (MaxPooling3 (None, 10, 15, 15, 8)     0         
_________________________________________________________________
time_distributed_5 (TimeDist (None, 10, 15, 15, 8)     0         
_________________________________________________________________
conv_lst_m2d_8 (ConvLSTM2D)  (None, 10, 13, 13, 14)   

In [20]:
early_stopping_callback = EarlyStopping(monitor = 'val_loss', patience = 10, mode = 'min', restore_best_weights = True)

convlstm_model.compile(loss = 'binary_crossentropy', optimizer = 'Adam', metrics = ["accuracy"])
convlstm_model_training_history = convlstm_model.fit(x = features_train, y = labels_train, epochs = 70, batch_size = 4,
                                                     shuffle = True, validation_split = 0.25, 
                                                     callbacks = [early_stopping_callback])

Epoch 1/70
150/150 [==============================] - 78s 467ms/step - loss: 0.5253 - accuracy: 0.7283 - val_loss: 0.3854 - val_accuracy: 0.8200
Epoch 2/70
150/150 [==============================] - 76s 505ms/step - loss: 0.3111 - accuracy: 0.8833 - val_loss: 0.3307 - val_accuracy: 0.8667
Epoch 3/70
150/150 [==============================] - 79s 527ms/step - loss: 0.2742 - accuracy: 0.9150 - val_loss: 0.3598 - val_accuracy: 0.8467
Epoch 4/70
150/150 [==============================] - 70s 465ms/step - loss: 0.2322 - accuracy: 0.9100 - val_loss: 0.3468 - val_accuracy: 0.8733
Epoch 5/70
150/150 [==============================] - 64s 428ms/step - loss: 0.2184 - accuracy: 0.9200 - val_loss: 0.3770 - val_accuracy: 0.8800
Epoch 6/70
150/150 [==============================] - 64s 425ms/step - loss: 0.2021 - accuracy: 0.9300 - val_loss: 0.3465 - val_accuracy: 0.8667
Epoch 7/70
150/150 [==============================] - 64s 426ms/step - loss: 0.1705 - accuracy: 0.9483 - val_loss: 0.3719 - val_ac

In [21]:
model_evaluation_history = convlstm_model.evaluate(features_test, labels_test)

8/8 [==============================] - 3s 400ms/step - loss: 0.2615 - accuracy: 0.9040


In [16]:
y_pred = convlstm_model.predict_classes(features_test)
new_array = to_categorical(y_pred,2)


from sklearn.metrics import confusion_matrix
print(confusion_matrix(labels_test.argmax(axis=1), new_array.argmax(axis=1)))

from sklearn.metrics import f1_score
print(f1_score(labels_test.argmax(axis=1), new_array.argmax(axis=1)))

[[112  13]
 [  7 118]]
0.921875


In [29]:
def plot_metric(model_training_history, metric_name_1, metric_name_2, plot_name):
  
    
    metric_value_1 = model_training_history.history[metric_name_1]
    metric_value_2 = model_training_history.history[metric_name_2]
    
    epochs = range(len(metric_value_1))

    plt.plot(epochs, metric_value_1, 'blue', label = metric_name_1)
    plt.plot(epochs, metric_value_2, 'red', label = metric_name_2)

    plt.title(str(plot_name))

    plt.legend()

In [ ]:
plot_metric(convlstm_model_training_history, 'loss', 'val_loss', 'Total Loss vs Total Validation Loss')

In [ ]:
plot_metric(convlstm_model_training_history, 'accuracy', 'val_accuracy', 'Total accuracy vs Total validation accuracy')